In [1]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from statsmodels.robust.scale import huber
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import re
import locale
import geopandas
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)


# df_indicadores_pob = pd.read_parquet(f'../data/processed/df_indicadores_pob.parquet')
# df_final_pob = pd.read_parquet('../data/processed/df_final_pob.parquet')
# df_final_pob_melt = pd.read_parquet('../data/processed/df_final_pob_melt.parquet')
# df_final_pob_melt_PC = pd.read_parquet('../data/processed/df_final_pob_melt_PC.parquet')
# CCAA_CO = pd.read_parquet('../data/processed/CCAA_CO.parquet')
# MUNI_CO = pd.read_parquet('../data/processed/MUNI_CO.parquet')
# PROV_CO = pd.read_parquet('../data/processed/PROV_CO.parquet')



In [2]:
df_final_pob=pd.read_parquet('../data/main_processed/df_final_pob.parquet')

df_indicadores_pob=pd.read_parquet('../data/main_processed/df_indicadores_pob.parquet')

df_indicadores_pob_pivot=pd.read_parquet('../data/main_processed/df_indicadores_pob_pivot.parquet')

df_final_pob_melt=pd.read_parquet('../data/main_processed/df_final_pob_melt.parquet')

df_final_pob_melt_PC=pd.read_parquet('../data/main_processed/df_final_pob_melt_PC.parquet')

CCAA_CO = pd.read_parquet('../data/main_processed/CCAA_CO.parquet')
MUNI_CO = pd.read_parquet('../data/main_processed/MUNI_CO.parquet')
PROV_CO = pd.read_parquet('../data/main_processed/PROV_CO.parquet')

df_final_pob_dropdown=pd.read_parquet('../data/main_processed/df_final_pob_dropdown.parquet')

df_final_pob_dropdown_c=pd.read_parquet('../data/main_processed/df_final_pob_dropdown_c.parquet')

df_final_pob_poblaciontext=pd.read_parquet('../data/main_processed/df_final_pob_poblaciontext.parquet')




In [8]:
CCAA_types='Andalucía'

In [9]:
df_count_c=df_final_pob.pivot_table(index=['CCAA'] ,values=['TOTAL','Población 2018'] ,aggfunc=sum).reset_index()
df_count_c['PC_TOTAL'] = df_count_c.apply(lambda new: round(new['TOTAL']/new['Población 2018'],), axis=1)
df_count_c=df_count_c.sort_values(by='PC_TOTAL',ascending=False)
df_count_c['CCAA'] = df_count_c['CCAA'].astype('object')


df_count_c_new_n=df_count_c.loc[(df_count_c['PC_TOTAL']==df_count_c['PC_TOTAL'].quantile(0.75, interpolation='nearest'))|\
                (df_count_c['PC_TOTAL']==df_count_c['PC_TOTAL'].quantile(0.50, interpolation='nearest'))|\
                 (df_count_c['PC_TOTAL']==df_count_c['PC_TOTAL'].quantile(0.25, interpolation='nearest'))]


df = df_count_c_new_n
df = df.append(df_count_c.loc[df_count_c['CCAA']==CCAA_types])

In [10]:
df

,CCAA,Población 2018,TOTAL,PC_TOTAL
6,Cataluña,7271555.0,5.646433e+09,777
13,Principado de Asturias,987025.0,6.729865e+08,682
4,Castilla y León,1947252.0,1.190997e+09,612
0,Andalucía,7677889.0,5.554223e+09,723


In [22]:
df_count_p=df_final_pob.pivot_table(index=['Provincia'] ,values=['TOTAL','Población 2018'] ,aggfunc=sum).reset_index()
df_count_p['PC_TOTAL'] = df_count_p.apply(lambda new: round(new['TOTAL']/new['Población 2018'],), axis=1)
df_count_p['Provincia'] = df_count_p['Provincia'].astype('object')

df_count_p_new_n=df_count_p.loc[(df_count_p['PC_TOTAL']==df_count_p['PC_TOTAL'].quantile(0.75, interpolation='nearest'))|\
                (df_count_p['PC_TOTAL']==df_count_p['PC_TOTAL'].quantile(0.51, interpolation='nearest'))|\
                 (df_count_p['PC_TOTAL']==df_count_p['PC_TOTAL'].quantile(0.25, interpolation='nearest'))]


In [23]:
df_count_p_new_n

,Provincia,Población 2018,TOTAL,PC_TOTAL
22,Jaén,549573.0,3.239394e+08,589
24,Lleida,377734.0,2.820167e+08,747
36,Segovia,118354.0,7.733025e+07,653


In [24]:
cohorte='más de 500.000'
municipio_types = 'Madrid'



In [44]:
cohorte = df_final_pob.loc[df_final_pob['Nombre Ente Principal'] == municipio_types , 'cohorte_pob'].unique().to_list()[0]

df = df_final_pob[['Nombre Ente Principal' , 'cohorte_pob' , 'PC_TOTAL']].loc[
    (df_final_pob['cohorte_pob'] == cohorte) & (df_final_pob['PC_TOTAL'] > 0)].sort_values(by='PC_TOTAL' ,
                                                                                           ascending=False)
df['Nombre Ente Principal'] = df['Nombre Ente Principal'].astype('object')
# df['PC_TOTAL'] = df.apply(lambda new: round(new['PC_TOTAL'] , ) , axis=1)
df2=df.loc[(df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.75, interpolation='nearest'))|\
                (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.50, interpolation='nearest'))|\
                 (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.25, interpolation='nearest'))]




df2 = df2.append(df.loc[df['Nombre Ente Principal']==municipio_types])
df=df2
df.iloc[3 , 0] = f'{municipio_types}.'




In [45]:
df

,Nombre Ente Principal,cohorte_pob,PC_TOTAL
645,Sevilla,más de 500.000,994.624372
2825,Barcelona,más de 500.000,872.550323
1115,Zaragoza,más de 500.000,863.149121
4186,Madrid.,más de 500.000,1118.909850


In [47]:
partida_de_coste_types='Abastecimiento de agua potable'

In [50]:
pob_c=df_final_pob.pivot_table(index=['CCAA'] ,values=['Población 2018'] ,aggfunc=sum).reset_index()
df_count_c_pc=df_final_pob_melt.pivot_table(index=['CCAA','Descripción'] ,values=['coste_efectivo'] ,aggfunc=sum).reset_index()
df_count_c_pc = pd.merge(df_count_c_pc, pob_c, on='CCAA', how='left')
df_count_c_pc['PC_TOTAL'] = df_count_c_pc.apply(lambda new: new['coste_efectivo']/new['Población 2018'], axis=1)
df_count_c_pc['CCAA'] = df_count_c_pc['CCAA'].astype('object')


df = df_count_c_pc

df = df.loc[df['Descripción'] == partida_de_coste_types].sort_values(by='PC_TOTAL' ,ascending=False)


df =df.loc[(df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.90, interpolation='nearest'))|\
                (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.75, interpolation='nearest'))|\
                 (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.25, interpolation='nearest'))|\
                 (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.50, interpolation='nearest'))]
df['PC_TOTAL'] = df.apply(lambda new: round(new['PC_TOTAL'],), axis=1)




df 





,CCAA,Descripción,coste_efectivo,Población 2018,PC_TOTAL
86,Canarias,Abastecimiento de agua potable,1.523355e+08,1932728.0,79
430,Galicia,Abastecimiento de agua potable,1.156006e+08,2335771.0,49
43,Aragón,Abastecimiento de agua potable,5.280455e+07,1235915.0,43
559,Principado de Asturias,Abastecimiento de agua potable,4.024742e+07,987025.0,41


In [52]:
PROV_types='Murcia'

In [56]:
pob_p=df_final_pob.pivot_table(index=['Provincia'] ,values=['Población 2018'] ,aggfunc=sum).reset_index()
df_count_p_pc=df_final_pob_melt.pivot_table(index=['Provincia','Descripción'] ,values=['coste_efectivo'] ,aggfunc=sum).reset_index()
df_count_p_pc = pd.merge(df_count_p_pc, pob_p, on='Provincia', how='left')
df_count_p_pc['PC_TOTAL'] = df_count_p_pc.apply(lambda new: new['coste_efectivo']/new['Población 2018'], axis=1)
df_count_p_pc['Provincia'] = df_count_p_pc['Provincia'].astype('object')

df = df_count_p_pc

df = df.loc[df['Descripción'] == partida_de_coste_types].sort_values(by='PC_TOTAL' , ascending=False)

df2 = df.loc[(df['PC_TOTAL'] == df['PC_TOTAL'].quantile(0.75 , interpolation='nearest')) | \
             (df['PC_TOTAL'] == df['PC_TOTAL'].quantile(0.50 , interpolation='nearest')) | \
             (df['PC_TOTAL'] == df['PC_TOTAL'].quantile(0.25 , interpolation='nearest'))]
df2 = df2.sort_values(by='PC_TOTAL' , ascending=False)

df2 = df2.append(df.loc[df['Provincia'] == PROV_types])
# df2['PC_TOTAL'] = df2.apply(lambda new: round(new['PC_TOTAL'] , ) , axis=1)
df2.iloc[3 , 0] = f'{PROV_types}.'



df2

,Provincia,Descripción,coste_efectivo,Población 2018,PC_TOTAL
428,Ciudad Real,Abastecimiento de agua potable,2.166432e+07,422487.0,51.278081
129,Asturias,Abastecimiento de agua potable,4.024742e+07,987025.0,40.776494
1582,Sevilla,Abastecimiento de agua potable,6.333921e+07,1775103.0,35.681994
1156,Murcia.,Abastecimiento de agua potable,1.517914e+08,1423404.0,106.639748


In [70]:
cohorte='0-1000' 
municipio_types = 'Abadía'
partida_de_coste_types='Acceso a los núcleos de población'


In [84]:
cohorte = df_final_pob_melt_PC.loc[df_final_pob_melt_PC['Nombre Ente Principal'] == municipio_types , 'cohorte_pob'].unique().to_list()[0]

df = df_final_pob_melt_PC[['Nombre Ente Principal' , 'cohorte_pob' , 'Descripción' , 'coste_efectivo_PC']].loc[
    (df_final_pob_melt_PC['cohorte_pob'] == cohorte)  & ( df_final_pob_melt_PC['Descripción'] == partida_de_coste_types)].sort_values(by='coste_efectivo_PC' , ascending=False)
df['Nombre Ente Principal'] = df['Nombre Ente Principal'].astype('object')

df2 = df.loc[(df['coste_efectivo_PC'] == df['coste_efectivo_PC'].quantile(0.75 , interpolation='nearest')) | \
             (df['coste_efectivo_PC'] == df['coste_efectivo_PC'].quantile(0.50 , interpolation='nearest')) | \
             (df['coste_efectivo_PC'] == df['coste_efectivo_PC'].quantile(0.25 , interpolation='nearest'))]

df2 = df2.sort_values(by='coste_efectivo_PC' , ascending=False)
df2.drop_duplicates(subset='coste_efectivo_PC' , keep='last',inplace=True)

df = df2.append(df.loc[df['Nombre Ente Principal'] == municipio_types])

df.iloc[3 , 0] = f'{municipio_types}.'
df['coste_efectivo_PC'] = df.apply(lambda new: round(new['coste_efectivo_PC'] , ) , axis=1)



In [85]:
df

,Nombre Ente Principal,cohorte_pob,Descripción,coste_efectivo_PC
10672,Bardallur,0-1.000,Acceso a los núcleos de población,29
11768,Alba de Yeltes,0-1.000,Acceso a los núcleos de población,6
13026,Estaràs,0-1.000,Acceso a los núcleos de población,1
13448,Abadía.,0-1.000,Acceso a los núcleos de población,43
